preprocessor_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

Detected remote with confidence 0.998 at location [40.16, 70.81, 175.55, 117.98]
Detected remote with confidence 0.996 at location [333.24, 72.55, 368.33, 187.66]
Detected couch with confidence 0.995 at location [-0.02, 1.15, 639.73, 473.76]
Detected cat with confidence 0.999 at location [13.24, 52.05, 314.02, 470.93]
Detected cat with confidence 0.999 at location [345.4, 23.85, 640.37, 368.72]


In [4]:
outputs

DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-19.1194,  -0.0893, -11.0154,  ...,  -8.7117,  -9.8543,  10.6531],
         [-17.3640,  -1.8035, -14.0219,  ...,  -8.2177,  -4.1631,  11.4021],
         [-20.0461,  -0.5837, -11.1060,  ...,  -9.2851, -12.1836,  11.1371],
         ...,
         [-19.7237,  -1.3058, -10.1437,  ..., -10.4849, -12.9238,  11.2945],
         [-12.8113,   1.9661,  -5.0197,  ...,  -6.7028,  -7.3127,   6.2644],
         [-17.3271,  -0.8608, -13.1640,  ...,  -6.6456,  -3.8822,  10.6797]]],
       grad_fn=<ViewBackward0>), pred_boxes=tensor([[[0.4433, 0.5302, 0.8853, 0.9056],
         [0.5494, 0.2517, 0.0529, 0.2015],
         [0.4998, 0.5360, 0.9956, 0.9024],
         [0.5534, 0.1835, 0.0415, 0.0616],
         [0.4998, 0.4116, 0.9948, 0.8237],
         [0.5957, 0.5033, 0.8091, 0.9647],
         [0.5416, 0.2791, 0.0393, 0.2099],
         [0.1607, 0.2160, 0.2274, 0.1434],
         [0.1738, 0.1822, 0.1956, 0.0785],
         [0.4997, 0.3887, 0.999

In [10]:

from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])


results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.1)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

Detected remote with confidence 0.998 at location [40.16, 70.81, 175.55, 117.98]
Detected remote with confidence 0.996 at location [333.24, 72.55, 368.33, 187.66]
Detected couch with confidence 0.995 at location [-0.02, 1.15, 639.73, 473.76]
Detected cat with confidence 0.999 at location [13.24, 52.05, 314.02, 470.93]
Detected cat with confidence 0.999 at location [345.4, 23.85, 640.37, 368.72]
